In [1]:
import numpy as np
import pandas as pd


# 교차검증

In [5]:
production_tb = pd.read_csv("./production.csv")

In [6]:
production_tb.head(3)


,type,length,thickness,fault_flg
0,E,274.027383,40.241131,False
1,D,86.319269,16.906715,False
2,E,123.940388,1.018462,False


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# 홀드아웃 검증을 위한 데이터 분할
# 예측 모델의 입력값과 예측 대상의 값을 각각 train_test_split함수에 설정
# test_size는 검증 데이터 분할

train_data, test_data, train_target, test_target = train_test_split(production_tb.drop("fault_flg", axis = 1), production_tb[["fault_flg"]], test_size = 0.2)

# train_test_split으로 행 이름ㅇ르 현재의 행 번호로 바꾼다

train_data.reset_index(inplace = True, drop = True)
test_data.reset_index(inplace = True, drop = True)
train_target.reset_index(inplace = True, drop = True)
test_target.reset_index(inplace = True, drop = True)

In [11]:
# 대상 행 번호 리스트 생성
row_no_list = list(range(len(train_target)))

In [12]:
# 교차 검증을 위한 데이터로 분할

k_fold = KFold(n_splits=4, shuffle = True)


In [13]:
# 교차수만큼 반복 처리.

for train_cv_no, test_cv_no in k_fold.split(row_no_list):
    
    train_cv = train_data.iloc[train_cv_no, :]
    test_cv = train_data.iloc[test_cv_no, :]
    

In [14]:
train_cv

,type,length,thickness
0,D,185.321841,6.087064
1,D,126.517115,3.932203
2,C,173.263979,15.627559
3,A,112.379238,8.088357
7,A,92.955242,6.848691
...,...,...,...
795,A,97.665450,0.782271
796,C,211.641260,34.440495
797,D,212.243506,4.676826
798,C,225.696466,37.275177


In [15]:
test_cv

,type,length,thickness
4,B,245.772723,26.228741
5,D,103.298048,7.292837
6,C,344.978431,15.644446
9,B,296.109080,31.477927
10,E,118.438979,6.265794
...,...,...,...
775,D,92.315788,18.346373
778,E,186.891376,4.784968
788,A,84.287272,14.615888
790,A,180.464678,7.366918


# 학습 및 검증을 위한 시간 데이터 준비

## 파이썬에는 시간 데이터 분할을 쉽게 구현할 수 있는 라이브러리 없음

In [17]:
train_window_start = 1 # 학습 데이터의 시작 행 번호
train_winodw_end = 24 # 종료 행 번호

horizon = 12 # 검증 데이터의 데이터수 지정

skip = 12 # skip에 이동할 데이터 수 설정

monthly_index_tb = pd.read_csv("monthly_index.csv")
monthly_index_tb.head()

,year_month,sales_amount,customer_number
0,2010-01,7191240,6885
1,2010-02,6253663,6824
2,2010-03,6868320,7834
3,2010-04,7147388,8552
4,2010-05,8755929,8171


In [18]:
# 연 월을 기준으로 데이터 정렬
monthly_index_tb.sort_values(by = "year_month")

,year_month,sales_amount,customer_number
0,2010-01,7191240,6885
1,2010-02,6253663,6824
2,2010-03,6868320,7834
3,2010-04,7147388,8552
4,2010-05,8755929,8171
...,...,...,...
115,2019-08,80528792,97059
116,2019-09,90191653,86560
117,2019-10,95749954,99507
118,2019-11,86998004,85215


In [19]:
while True:
    test_window_end = train_winodw_end + horizon
    
    train = monthly_index_tb[train_window_start : train_winodw_end]
    
    test = monthly_index_tb[train_window_start : train_winodw_end]
    
    if test_window_end >= len(monthly_index_tb.index):
        break
        
    train_window_start += skip
    train_winodw_end += skip

In [20]:
train_window_start

85

In [21]:
train_winodw_end

108